# Resample Fruad=1 Class
- Train using Keras DNN

In [1]:
import pandas as pd
import numpy as np
import keras

from collections import Counter

from sklearn.utils import resample, shuffle

from keras.layers import Dense, Dropout, BatchNormalization

from sklearn.metrics import roc_auc_score

from multiprocessing import cpu_count

from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE

import xgboost as xgb

!python -c 'import tensorflow as tf; print(tf.version)'
print(cpu_count())

Using TensorFlow backend.


<module 'tensorflow._api.v1.version' from '/home/ec2-user/anaconda3/lib/python3.7/site-packages/tensorflow/_api/v1/version/__init__.py'>
2


# Read Dataset

In [2]:
def get_string_features(df):
    string_features = []
    for col in df.columns:
        if df[col].dtype == np.dtype('object'):
            string_features.append(col)

    return string_features


def read_data(filename='./datasets/kfold/transaction_fold_0_0_0.csv', drop_string_features=True):
    
    df = pd.read_csv(filename)
    
    if drop_string_features:
        string_features = get_string_features(df)
        df = df.drop(columns=string_features)
    
    return df

In [3]:
train = read_data(drop_string_features=True)

In [4]:
train.shape

(73816, 380)

In [5]:
train.isFraud.value_counts()

0    71376
1     2440
Name: isFraud, dtype: int64

# Undersample majority class

In [6]:
def preprocess(df):
    df = df.fillna(-999)
    
    return df.drop(columns=['TransactionID', 'TransactionDT'])


train = preprocess(train)

In [7]:
train.shape

(73816, 378)

In [8]:
def split_features_labels(df):
    return df.drop(columns=['isFraud']), df['isFraud']

In [9]:
def undersample_then_oversample(df, random_state=27):
    """
    Upsample minority class (isFraud=1), combine with majority class, and then shuffle them.
    """
    sampling_strategy = 0.5
#     X = df.drop(columns=['isFraud'])
#     y = df['isFraud']
    X, y = split_features_labels(df)
    
    # Declare Random Under Sampler
    rus = NearMiss(version=3, 
                   sampling_strategy=sampling_strategy,
                   n_jobs=cpu_count(),
                   random_state=random_state)
    
    ros = SMOTE(n_jobs=cpu_count(),
               random_state=random_state)
    
    X, y = rus.fit_resample(X, y)
    X, y = ros.fit_resample(X, y)
    
    return X, y.astype('int32')


%time X, y = undersample_then_oversample(train)

CPU times: user 31.7 s, sys: 652 ms, total: 32.3 s
Wall time: 24.3 s


In [10]:
print(X.shape)
print(y.shape)

(9760, 377)
(9760,)


In [11]:
Counter(y)

Counter({0: 4880, 1: 4880})

# Build Model

In [29]:
def get_model(input_dim, lr=1e-3):
    
    inputs = keras.layers.Input(shape=[input_dim,])
    
    x_attention = keras.layers.Dense(input_dim, activation='softmax', kernel_initializer = 'uniform')(inputs)
    x = keras.layers.concatenate([x_attention, inputs])
    
    hidden_size = 256
    x = keras.layers.Dense(hidden_size, activation='relu', kernel_initializer = 'uniform')(x)
    x_attention = keras.layers.Dense(hidden_size, activation='softmax', kernel_initializer = 'uniform')(x)
    x = keras.layers.concatenate([x_attention, x])
    
    
    hidden_size = 128
    x = keras.layers.Dense(hidden_size, activation='relu', kernel_initializer = 'uniform')(x)
    x_attention = keras.layers.Dense(hidden_size, activation='softmax', kernel_initializer = 'uniform')(x)
    x = keras.layers.concatenate([x_attention, x])
    
    
    hidden_size = 64
    x = keras.layers.Dense(hidden_size, activation='relu', kernel_initializer = 'uniform')(x)
    x_attention = keras.layers.Dense(hidden_size, activation='softmax', kernel_initializer = 'uniform')(x)
    x = keras.layers.concatenate([x_attention, x])
    
    hidden_size = 32
    x = keras.layers.Dense(hidden_size, activation='relu', kernel_initializer = 'uniform')(x)
    x_attention = keras.layers.Dense(hidden_size, activation='softmax', kernel_initializer = 'uniform')(x)
    x = keras.layers.concatenate([x_attention, x])
    
    
    hidden_size = 16
    x = keras.layers.Dense(hidden_size, activation='relu', kernel_initializer = 'uniform')(x)
    x_attention = keras.layers.Dense(hidden_size, activation='softmax', kernel_initializer = 'uniform')(x)
    x = keras.layers.concatenate([x_attention, x])
    
    
    hidden_size = 4
    x = keras.layers.Dense(hidden_size, activation='relu', kernel_initializer = 'uniform')(x)
    x_attention = keras.layers.Dense(hidden_size, activation='softmax', kernel_initializer = 'uniform')(x)
    x = keras.layers.concatenate([x_attention, x])
    
    
    outputs = keras.layers.Dense(1, activation='sigmoid')(x)
    
    model = keras.models.Model(inputs=inputs,
                              outputs=outputs)
    
    
    optimizer = keras.optimizers.Adam(lr=lr)

    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                 metrics=['accuracy'])
    
    
    return model


model = get_model(X.shape[1], lr=1e-3)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 377)          0                                            
__________________________________________________________________________________________________
dense_67 (Dense)                (None, 377)          142506      input_12[0][0]                   
__________________________________________________________________________________________________
concatenate_17 (Concatenate)    (None, 754)          0           dense_67[0][0]                   
                                                                 input_12[0][0]                   
__________________________________________________________________________________________________
dense_68 (Dense)                (None, 256)          193280      concatenate_17[0][0]             
__________

# Train

In [38]:
epochs = 500
batch_size = 256

model.fit(X, y,
         epochs=epochs,
         batch_size=batch_size)

Epoch 1/500
9760/9760 [==============================] - 1s 72us/step - loss: 0.1942 - acc: 0.9127
Epoch 2/500
9760/9760 [==============================] - 1s 74us/step - loss: 0.2157 - acc: 0.9012
Epoch 3/500
9760/9760 [==============================] - 1s 72us/step - loss: 0.2246 - acc: 0.9002
Epoch 4/500
9760/9760 [==============================] - 1s 73us/step - loss: 0.2229 - acc: 0.9018
Epoch 5/500
9760/9760 [==============================] - 1s 72us/step - loss: 0.2218 - acc: 0.9051
Epoch 6/500
9760/9760 [==============================] - 1s 72us/step - loss: 0.2501 - acc: 0.8944
Epoch 7/500
9760/9760 [==============================] - 1s 71us/step - loss: 0.2129 - acc: 0.9105
Epoch 8/500
9760/9760 [==============================] - 1s 72us/step - loss: 0.1951 - acc: 0.9172
Epoch 9/500
9760/9760 [==============================] - 1s 75us/step - loss: 0.1952 - acc: 0.9187
Epoch 10/500
9760/9760 [==============================] - 1s 74us/step - loss: 0.1888 - acc: 0.9195
Epoch 11/

9760/9760 [==============================] - 1s 68us/step - loss: 0.1719 - acc: 0.9172
Epoch 84/500
9760/9760 [==============================] - 1s 70us/step - loss: 0.1775 - acc: 0.9156
Epoch 85/500
9760/9760 [==============================] - 1s 69us/step - loss: 0.1967 - acc: 0.9085
Epoch 86/500
9760/9760 [==============================] - 1s 69us/step - loss: 0.1606 - acc: 0.9252
Epoch 87/500
9760/9760 [==============================] - 1s 68us/step - loss: 0.1819 - acc: 0.9115
Epoch 88/500
9760/9760 [==============================] - 1s 70us/step - loss: 0.1967 - acc: 0.9047
Epoch 89/500
9760/9760 [==============================] - 1s 70us/step - loss: 0.1683 - acc: 0.9190
Epoch 90/500
9760/9760 [==============================] - 1s 70us/step - loss: 0.1729 - acc: 0.9170
Epoch 91/500
9760/9760 [==============================] - 1s 72us/step - loss: 0.1645 - acc: 0.9205
Epoch 92/500
9760/9760 [==============================] - 1s 71us/step - loss: 0.1891 - acc: 0.9113
Epoch 93/500


9760/9760 [==============================] - 1s 72us/step - loss: 0.2193 - acc: 0.9178
Epoch 165/500
9760/9760 [==============================] - 1s 72us/step - loss: 0.2335 - acc: 0.9136
Epoch 166/500
9760/9760 [==============================] - 1s 72us/step - loss: 0.2150 - acc: 0.9227
Epoch 167/500
9760/9760 [==============================] - 1s 72us/step - loss: 0.2322 - acc: 0.9127
Epoch 168/500
9760/9760 [==============================] - 1s 73us/step - loss: 0.2378 - acc: 0.9109
Epoch 169/500
9760/9760 [==============================] - 1s 73us/step - loss: 0.2151 - acc: 0.9219
Epoch 170/500
9760/9760 [==============================] - 1s 71us/step - loss: 0.2079 - acc: 0.9261
Epoch 171/500
9760/9760 [==============================] - 1s 70us/step - loss: 0.2313 - acc: 0.9102
Epoch 172/500
9760/9760 [==============================] - 1s 70us/step - loss: 0.2261 - acc: 0.9156
Epoch 173/500
9760/9760 [==============================] - 1s 70us/step - loss: 0.2037 - acc: 0.9254
Epoc

9760/9760 [==============================] - 1s 70us/step - loss: 0.2167 - acc: 0.9241
Epoch 246/500
9760/9760 [==============================] - 1s 71us/step - loss: 0.2078 - acc: 0.9295
Epoch 247/500
9760/9760 [==============================] - 1s 74us/step - loss: 0.2450 - acc: 0.9133
Epoch 248/500
9760/9760 [==============================] - 1s 72us/step - loss: 0.2146 - acc: 0.9256
Epoch 249/500
9760/9760 [==============================] - 1s 70us/step - loss: 0.2189 - acc: 0.9234
Epoch 250/500
9760/9760 [==============================] - 1s 71us/step - loss: 0.2276 - acc: 0.9196
Epoch 251/500
9760/9760 [==============================] - 1s 70us/step - loss: 0.2023 - acc: 0.9305
Epoch 252/500
9760/9760 [==============================] - 1s 70us/step - loss: 0.2212 - acc: 0.9235
Epoch 253/500
9760/9760 [==============================] - 1s 68us/step - loss: 0.2298 - acc: 0.9208
Epoch 254/500
9760/9760 [==============================] - 1s 69us/step - loss: 0.2425 - acc: 0.9130
Epoc

9760/9760 [==============================] - 1s 69us/step - loss: 0.2085 - acc: 0.9254
Epoch 327/500
9760/9760 [==============================] - 1s 71us/step - loss: 0.2340 - acc: 0.9167
Epoch 328/500
9760/9760 [==============================] - 1s 71us/step - loss: 0.2379 - acc: 0.9141
Epoch 329/500
9760/9760 [==============================] - 1s 71us/step - loss: 0.2156 - acc: 0.9249
Epoch 330/500
9760/9760 [==============================] - 1s 71us/step - loss: 0.2024 - acc: 0.9268
Epoch 331/500
9760/9760 [==============================] - 1s 72us/step - loss: 0.2338 - acc: 0.9148
Epoch 332/500
9760/9760 [==============================] - 1s 72us/step - loss: 0.2147 - acc: 0.9234
Epoch 333/500
9760/9760 [==============================] - 1s 70us/step - loss: 0.1970 - acc: 0.9297
Epoch 334/500
9760/9760 [==============================] - 1s 69us/step - loss: 0.1921 - acc: 0.9300
Epoch 335/500
9760/9760 [==============================] - 1s 72us/step - loss: 0.1800 - acc: 0.9346
Epoc

9760/9760 [==============================] - 1s 69us/step - loss: 0.1932 - acc: 0.9301
Epoch 408/500
9760/9760 [==============================] - 1s 70us/step - loss: 0.2005 - acc: 0.9293
Epoch 409/500
9760/9760 [==============================] - 1s 68us/step - loss: 0.1936 - acc: 0.9288
Epoch 410/500
9760/9760 [==============================] - 1s 69us/step - loss: 0.1856 - acc: 0.9321
Epoch 411/500
9760/9760 [==============================] - 1s 69us/step - loss: 0.3012 - acc: 0.8874
Epoch 412/500
9760/9760 [==============================] - 1s 70us/step - loss: 0.2119 - acc: 0.9217
Epoch 413/500
9760/9760 [==============================] - 1s 70us/step - loss: 0.1813 - acc: 0.9352
Epoch 414/500
9760/9760 [==============================] - 1s 70us/step - loss: 0.1863 - acc: 0.9351
Epoch 415/500
9760/9760 [==============================] - 1s 69us/step - loss: 0.1794 - acc: 0.9361
Epoch 416/500
9760/9760 [==============================] - 1s 71us/step - loss: 0.1927 - acc: 0.9283
Epoc

9760/9760 [==============================] - 1s 69us/step - loss: 0.1781 - acc: 0.9349
Epoch 489/500
9760/9760 [==============================] - 1s 68us/step - loss: 0.1801 - acc: 0.9323
Epoch 490/500
9760/9760 [==============================] - 1s 69us/step - loss: 0.2058 - acc: 0.9199
Epoch 491/500
9760/9760 [==============================] - 1s 69us/step - loss: 0.1957 - acc: 0.9262
Epoch 492/500
9760/9760 [==============================] - 1s 69us/step - loss: 0.2075 - acc: 0.9223
Epoch 493/500
9760/9760 [==============================] - 1s 69us/step - loss: 0.1957 - acc: 0.9257
Epoch 494/500
9760/9760 [==============================] - 1s 69us/step - loss: 0.1859 - acc: 0.9310
Epoch 495/500
9760/9760 [==============================] - 1s 67us/step - loss: 0.2367 - acc: 0.9123
Epoch 496/500
9760/9760 [==============================] - 1s 69us/step - loss: 0.1680 - acc: 0.9385
Epoch 497/500
9760/9760 [==============================] - 1s 69us/step - loss: 0.2184 - acc: 0.9182
Epoc

In [39]:
pred_prob = model.predict(X,
                               batch_size=batch_size)

score = roc_auc_score(y, pred_prob)

print('roc-auc score={}'.format(score))


roc-auc score=0.9862151261421661


# Read Test Data

In [40]:
del train

import gc
gc.collect()

223

In [41]:
test = read_data('./datasets/kfold/transaction_fold_0_0_1.csv')

test = preprocess(test)

In [42]:
X, y = split_features_labels(test)

In [43]:
type(X)

pandas.core.frame.DataFrame

In [44]:
X = X.values

In [45]:
type(X)

numpy.ndarray

In [48]:
pred_prob = model.predict(X,
                         batch_size=batch_size)

score = roc_auc_score(y, pred_prob)

print('roc-auc score={}'.format(score))

roc-auc score=0.7333243170453261
